## Data analysis
https://www.r-graph-gallery.com/101_Manhattan_plot.html

## Running sos notebook

On the Yale Farnam cluster,


On a local computer with for example 8 threads,

```
sos run BOLT-LMM_R_analysis_UKB.ipynb -q none -j 8
```


In [52]:
[global]
# Working directory
parameter: cwd = path('/Users/dianacornejo/pleiotropy_UKB/data')
#Input individuals summary stats file
parameter: infile = 'chr?.gz'
#Output merged summary stats file
parameter: name = 'test'
#Ouput Manhattan plot
parameter: manhattan= 'BMI_manhattan'

Merge summary stats files using python

In [69]:
#[step_1 (convert)]
#The stats file contains all the summary statistics derived from the association analysis in BOLT
#input: 'chr?.gz'
cwd = path('/Users/dianacornejo/pleiotropy_UKB/data')
name = 'test'
infile = 'chr?.gz'
input: f'{cwd}/{infile}'
output: f'{cwd}/{name}'
python: expand=True
    import glob, os, gzip, shutil
    os.chdir('{cwd}')
    file_list = [f for f in glob.glob('{_input}')]
    file_list.sort()
    with open('test','wb') as outfile:
        for i, fname in enumerate(file_list):
            with gzip.open(fname, 'rb') as infile:
                if i !=0:
                    infile.readline()
                shutil.copyfileobj(infile, outfile)
                print(fname + " has been imported.")

In [ ]:
import glob, os, gzip
os.chdir('/Users/dianacornejo/pleiotropy_UKB/data')
file_list = [f for f in glob.glob('file?.gz')]
file_list.sort()
with open('test','w') as outfile:
    with gzip.open(file_list[0], 'rb') as f:
        for line in f:        #keep the header from file1
            outfile.write(line.decode('utf-8'))
    for file in (file_list[1:]):
        with gzip.open(x) as f:
            for line in f:
                if not line.startswith(b'SNP'):
                        outfile.write(line.decode('utf-8'))

In [7]:
file_list

['file3.gz', 'file1.gz', 'file4.gz']

In [24]:
with gzip.open('file4.gz','r') as f:
    for line in f:
        print(line.decode('utf-8'))
        

SNP	CHR	BP	GENPOS	ALLELE1	ALLELE0	A1FREQ	F_MISS	CHISQ_LINREG	P_LINREG	BETA	SE	CHISQ_BOLT_LMM_INF	P_BOLT_LMM_INF	CHISQ_BOLT_LMM	P_BOLT_LMM

rs3131962	1	756604	0.00490722	A	G	0.129619	0.00154251	0.143797	7.0E-01	-0.000595741	0.00322443	0.0341358	8.5E-01	0.00862527	9.3E-01

rs12562034	1	768448	0.00495714	A	G	0.105098	0.00160077	3.20665	7.3E-02	0.00652097	0.00352459	3.423	6.4E-02	2.74284	9.8E-02

rs4040617	1	779322	0.00500708	G	A	0.127412	0.00241641	0.681178	4.1E-01	-0.00195123	0.00324907	0.360661	5.5E-01	0.294739	5.9E-01

rs79373928	1	801536	0.0058722	G	T	0.0148708	0.000596473	0.00918373	9.2E-01	-0.00591515	0.00893767	0.438009	5.1E-01	0.358728	5.5E-01

rs11240779	1	808631	0.00620827	G	A	0.224724	0.00698567	1.30599	2.5E-01	0.00347077	0.00260351	1.77719	1.8E-01	1.59823	2.1E-01

rs57181708	1	809876	0.00620827	G	A	0.100445	0.000873903	6.64697	9.9E-03	0.00875674	0.00359352	5.93806	1.5E-02	5.17968	2.3E-02

rs4970383	1	838555	0.00620827	A	C	0.245613	0.00178665	0.43358	5.1E-01	0.000175294	0.00251

In [26]:
sum(1 for i in open('test', 'r'))

28

In [4]:
sum(1 for i in open(file_list[0], 'rb'))

80874

In [30]:
sum(1 for i in open('BMI_summary_stats.txt', 'r'))

904226

# First of all set the working directory. This is where the BOLT-LMM association results are located.

In [ ]:
[step_2 (plot)]
output: f'{out_dir}/{manhattan}'
R: expand=True
    install.packages("qqman")
    library(qqman)
    summary_gz=gzfile('{_name}.gz','rt')
    df=read.table('{_name}', header=TRUE,sep = ' ')
    png('{_manhattan}.png', width = 6, height = 4, unit='in', res=300)
    bmi_manhattan <- manhattan(df, chr='CHR', bp='BP', snp='SNP', p='P')
    dev.off()

Install and load required libraries

In [ ]:
#install.packages("reader")
#intall.packages("plyr")
#install.packages("ggplot2")
#library(readr)
#library(plyr)
#library(ggplot2)

Then create a list of files to be combined using R. In this case the summary statistics for the association analysis and then create the dataset by binding files from chr{1:22}. The drawback is that R runs out of memory when doing this

In [ ]:
#file_list = Sys.glob("*.snp_stats.bgen.gz") #one way of creating lists
#file_list = list.files(path=mydir, pattern="*.snp_stats.bgen.gz", full.names=TRUE) #another way of creating lists
#data.list = lapply(file_list, function(x){read.table(file = x,header = TRUE, sep = "\t")})
#data.merged = do.call("rbind", data.list

## Drawing the Manhattan Plot

In [ ]:

# Let's highlight them, with a bit of customization on the plot
#manhattan(gwasResults, highlight = snpsOfInterest)
#Annotate the SNPs of interest
#manhattan(gwasResults, annotatePval = 0.01)
#Make a qqplot
#qq(gwasResults$P)

In [35]:
import shutil
import glob

#import csv files from folder
os.chdir('/Users/dianacornejo/pleiotropy_UKB/pleiotropy_data/BMI')
allFiles = [f for f in glob.glob('*.snp_stats.bgen.gz')]
allFiles.sort()  # glob lacks reliable ordering, so impose your own if output order matters
with open('test', 'wb') as outfile:
    for i, fname in enumerate(allFiles):
        with gzip.open(fname, 'rb') as infile:
            if i != 0:
                infile.readline()  # Throw away header on all but first file
            # Block copy rest of file from input to output without parsing
            shutil.copyfileobj(infile, outfile)
            print(fname + " has been imported.")

ukb_imp_chr10_v3.UKB_caucasians_BMI_nopreg_adjagesex_residuals_andstandardized_022720.BoltLMM.snp_stats.bgen.gz has been imported.
ukb_imp_chr11_v3.UKB_caucasians_BMI_nopreg_adjagesex_residuals_andstandardized_022720.BoltLMM.snp_stats.bgen.gz has been imported.
ukb_imp_chr12_v3.UKB_caucasians_BMI_nopreg_adjagesex_residuals_andstandardized_022720.BoltLMM.snp_stats.bgen.gz has been imported.
ukb_imp_chr13_v3.UKB_caucasians_BMI_nopreg_adjagesex_residuals_andstandardized_022720.BoltLMM.snp_stats.bgen.gz has been imported.
ukb_imp_chr14_v3.UKB_caucasians_BMI_nopreg_adjagesex_residuals_andstandardized_022720.BoltLMM.snp_stats.bgen.gz has been imported.
ukb_imp_chr15_v3.UKB_caucasians_BMI_nopreg_adjagesex_residuals_andstandardized_022720.BoltLMM.snp_stats.bgen.gz has been imported.
ukb_imp_chr16_v3.UKB_caucasians_BMI_nopreg_adjagesex_residuals_andstandardized_022720.BoltLMM.snp_stats.bgen.gz has been imported.
ukb_imp_chr17_v3.UKB_caucasians_BMI_nopreg_adjagesex_residuals_andstandardized_0227

In [36]:
with open('test') as f:
   for line in f:
     print(line)
     break

SNP	CHR	BP	GENPOS	ALLELE1	ALLELE0	A1FREQ	INFO	CHISQ_LINREG	P_LINREG	BETA	SE	CHISQ_BOLT_LMM_INF	P_BOLT_LMM_INF	CHISQ_BOLT_LMM	P_BOLT_LMM



In [39]:
N = 10
with open("test") as file:  # the a opens it in append mode
    for i in range(N):
        line = next(file).strip()
        print(line)

SNP	CHR	BP	GENPOS	ALLELE1	ALLELE0	A1FREQ	INFO	CHISQ_LINREG	P_LINREG	BETA	SE	CHISQ_BOLT_LMM_INF	P_BOLT_LMM_INF	CHISQ_BOLT_LMM	P_BOLT_LMM
rs112920234	10	60523	-1.52522e-05	T	G	0.999862	0.842394	0.370499	5.4E-01	0.661995	0.865951	0.584418	4.4E-01	0.554502	4.6E-01
rs183305313	10	61334	-1.42418e-05	G	A	0.995398	0.84669	0.102032	7.5E-01	-0.0383797	0.0827471	0.215129	6.4E-01	0.134302	7.1E-01
rs568474105	10	62010	-1.33996e-05	C	T	0.999902	0.851176	0.900704	3.4E-01	0.982192	0.983597	0.997145	3.2E-01	0.938148	3.3E-01
rs12260013	10	66326	-8.02231e-06	A	G	0.972944	0.818375	1.46592	2.3E-01	-0.0665433	0.0351432	3.5853	5.8E-02	3.33519	6.8E-02
rs9943471	10	67774	-6.21826e-06	G	C	0.232007	0.802191	0.259476	6.1E-01	-0.000381117	0.0136921	0.000774777	9.8E-01	0.00906096	9.2E-01
rs35418599	10	69083	-4.58738e-06	C	T	0.259546	0.801306	0.328697	5.7E-01	0.00287088	0.0132044	0.0472709	8.3E-01	0.0851378	7.7E-01
rs61838967	10	71776	-1.23219e-06	T	C	0.187996	0.853084	0.248485	6.2E-01	0.00343529	0.0143574	0.0572496

In [42]:
sum(1 for i in open('test', 'r'))

16107521